# Activity 2: Basic high availability in Azure SQL Database

#### <i>The Azure SQL Workshop - Module 5</i>

<p style="border-bottom: 1px solid lightgrey;"></p>

In the module, you learned about the high availability architecture of Azure SQL. In this activity, you'll get to see how the General purpose tier of Azure SQL Database behaves similarly to a Failover Cluster Instance on-prem. The difference is that on-prem, this can be time-consuming or tricky to set up, but with Azure SQL, you get it out of the box.  

The activity will use the ostress tool you used in Module 4 to create a workload. You'll then initiate a failover using the Azure PowerShell module, and observe the effect it has on the ostress workload.  


**Set up**   

0. You should have opened this file using Azure Data Studio. If you didn't, please refer to Module 2 Activity 3 in the readme.md file to get set up.  
1. In the bar at the top of this screen, confirm or change the "Kernel" to **PowerShell**. This determines what language the code blocks in the file are. In this case, that language is PowerShell.  
2. Confirming the Kernel is **PowerShell**, for "Attach to", it should read **localhost**.  

Now that you're set up, you should read the text cells and "Run" the code cells by selecting the play button that appears in the left of a code cell when you hover over it.  


## Basic high availability (with no configuration!) in Azure SQL

In order to complete this activity, you will complete the following steps:

0. Confirm environment is properly configured  
1. Run the ostress workload  
1. Use PowerShell to initiate a failover  
1. Observe the results in ostress  
1. (Bonus) Look for signs in the portal that a failover occurred  

This notebook will guide you through getting ostress and a PowerShell notebook configured, and then you'll see how to use both together to initiate and analyze a failover of Azure SQL Database.  

>Note: Learn more about the PowerShell commands used and availabile [in the documentation](https://docs.microsoft.com/en-us/azure/sql-database/sql-database-powershell-samples?tabs=single-database).  

**Step 0 - Confirm environment is properly configured** 

In this step, there are 3 parts:  
* Part A: Configure keyboard shortcut  
* Part B: Confirm correct configuration of Az PowerShell Module  
* Part C: Confirm correct configuration of ostress  



**Part A: Configure keyboard shortcut**  


The following gif summarizes how to open Keyboard Shortcuts to the right. After the gif, there are step-by-step instructions.  

![](./graphics/shortcutgif.gif)   

First, in Azure Data Studio, go to **File** > **Preferences** > **Keyboard Shortcuts**.  

![](./graphics/keyboardshortcuts.png)  

Once this is open, you may find it easier to drag the **Keyboard Shortcuts** tab to the right, providing a split view.  

![](./graphics/splitview.png)  

Next, search for `terminal: run` and select the **+** button next to "Terminal: Run Selected Text in Active Terminal". select **CTRL** + **t** and then press **ENTER**.  

![](./graphics/ctrlt.png)  

You should now see `CTRL + T` under "Keybinding" for "Terminal: Run Selected Text in Active Terminal".   

![](./graphics/keyboardshortcuts2.png)  

You can now **close the "Keyboard Shortcuts" tab**, which will give you some screen real estate back.  

**Part B: Confirm correct configuration of Az PowerShell Module**


Confirm the [Az PowerShell Module](https://docs.microsoft.com/en-us/powershell/azure/install-az-ps?view=azps-3.4.00) is installed with version 3.4.0+. 

In [1]:
Get-InstalledModule -Name Az -AllVersions | select Name,Version


Name Version
---- -------
Az   3.5.0  




**Part C: Confirm correct configuration of ostress**

Confirm [RMUtils tool (which contains ostress)](https://www.microsoft.com/en-us/download/details.aspx?id=4511) is installed and has been added to the Path. 

In [2]:
ostress

OSTRESS, A Generic ODBC-Based Stress/Replay Utility.
Version 9.04.0051 built for x64.
Copyright ■ 1997-2014 Microsoft. All Rights Reserved
            Computer: win-vm01
         Base Module: C:\Program Files\Microsoft Corporation\RMLUtils\ostress.exe
          Process Id: 2188
 Active proc mask(0): 0x00000003
        Architecture: 9
           Page size: 4096
                CPUs: 2
    Processor groups: 1
        Highest node: 0
  Proximity: 00  Node: 00
---------------------------------------
               Group: 0
---------------------------------------
        Package mask: 0x00000003
        Processor(s): 0x00000003 Function units: Shared
        Processor(s): 0x00000003 assigned to Numa node: 0
Current time bias: 0 minutes 0.00 hours DST not used 
NOTE: All command line arguments are case sensitive.  No space is allowed between the parameter switch and its specified value.

USAGE:

  -S name of Microsoft SQL Server server to connect to
  -D ODBC data source name to use for conn

If either of the above cells fail, please revisit the prerequisites before continuing.  

In the below cell, replace the `0406` with the unique Workshop ID (e.g. `0406`, `168563`, etc.) you are using for the workshop. Also, add your subscription ID and your SQL authentication password for the logical server.   

> Note: If you have special characters in your password, you may need to escape them by placing a **`** before. For example, if my password is `$pass1w0rd`, I need to enter as **`$pass1w0rd**. Otherwise, you may have authentication issues in future steps.  

In [4]:
# Cell A
# Add your info
$unique_id = "0406"
$subscriptionId = "subscriptionID"
$password = "password"

# Do not modify
$admin = "cloudadmin"
$resourceGroup = "azuresqlworkshop$($unique_id)"
$server = "aw-server$($unique_id)"
$database = "AdventureWorks$($unique_id)"
$serverFullName = "$($server).database.windows.net"
Write-Host "Variables Received"

Variables Received


Now, run the following command to set the subscription within the notebook. 

In [ ]:
# Cell B
Set-AzContext -SubscriptionId $subscriptionId 

# DO NOT RUN THE NEXT CELL  
## READ ALL THE INSTRUCTIONS BEFORE DOING ANYTHING WITH THE NEXT CELL

**Step 1 - Run the ostress workload**  

The next step is to create a long-running workload so you can see how a failover affects the ability to read/write data, and how long a failover takes in the General purpose service tier for Azure SQL Database.  

The ostress workload below essentially connects and runs a simple query 50,000 times. Since this outputs essentially 50,000 results, running this in the Notebook is **not** a good idea. It will make the notebook huge, and you won't be able to run other cells (i.e. the failover command) while it is running.  

Instead, you will use an Azure Data Studio trick to run certain cells in the integrated terminal in Azure Data Studio. You already set up this shortcut in **Step 0**.    



Now things get interesting from a context-perspective. We want to run the ostress workload in the terminal, but the terminal maintains a separate context from the PowerShell notebook. That means the variables/configurations you set in the previous two cells, **have not been run in the terminal context**. In order to do this, for the two cells above, labeled **Cell A** and **Cell B** (with comments), click into the text. Then, select all the text (you can use `CTRL` + `a`) and use your new shortcut (use `CTRL` + `t`). You should see a terminal appear in the bottom section of your Azure Data Studio window, and you should see the results.  

Now, you will do the same for the following cell below: Select all the text (you can use `CTRL` + `a`) and use your new shortcut (use `CTRL` + `t`) to run the ostress workload in your integrated terminal. **Do not use the "Play" button to run this cell - use the shortcut**.   

If, at any time, you want to stop running the ostress workload before it is complete, you can go into the terminal and press `CTRL` + `c`.  

If, at any time, you want to run the workload again in the terminal, you can use the same method (select all the text and use the shortcut).  

> Note: If the ostress.exe command fails in the terminal, check your variables again. If you have special characters, you may need to escape them by placing a **`** before. For example, if my password is `$pass1w0rd`, I need to enter as **`$pass1w0rd**

In [ ]:
# Do not run in using the play button in the Notebook, select all and run in terminal with the shortcut created in the instructions.  
ostress.exe -S"$($serverFullName)" -Q"SELECT COUNT(*) FROM SalesLT.Customer" -U"$($admin)" -d"$($database)" -P"$($password)" -n1 -r50000

**Step 2 - Use PowerShell to initiate a failover**  

Run the following cell (using the "Play" button).

In [5]:
# create a failover
Invoke-AzSqlDatabaseFailover -ResourceGroupName $resourceGroup `
    -ServerName $server `
    -DatabaseName $database

**Step 3 - Observe the results in ostress**  

While this cell is running, you should observe any changes that appear in the terminal. You'll notice that while the failover occurs, for some time you cannot access the database. Once the failover completes, you can see the workload runs successfully again. The importance of retry logic in your application is very important, because if Azure decides to fail you over (for a number of reasons), you don't want your application to crash or become down for any longer than it takes for the failover to occur.  

This tool to create a failover on command can be useful in certain scenarios. It's important to note that the service does throttle you from doing this too often. Press "Play" in the cell below to try to run the failover again.  

In [7]:
# create a failover again
Invoke-AzSqlDatabaseFailover -ResourceGroupName $resourceGroup `
    -ServerName $server `
    -DatabaseName $database

Invoke-AzSqlDatabaseFailover : Long running operation failed with status 'Failed'. Additional Info:'There was a recent 
failover on the database or pool if database belongs in an elastic pool.  At least 30 minutes must pass between 
database failovers.'
At line:3 char:1
+ Invoke-AzSqlDatabaseFailover -ResourceGroupName $resourceGroup `
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : CloseError: (:) [Invoke-AzSqlDatabaseFailover], CloudException
    + FullyQualifiedErrorId : Microsoft.Azure.Commands.Sql.Database.Cmdlet.InvokeAzureSqlDatabaseFailover
 


You can now stop the workload by clicking on the terminal and selecting `CTRL` + `c`  

And you can close the terminal by clicking the **X** in the top-right corner of the terminal.  

## Save this file (File > Save or CTRL+S) for a future exercise.

**(Bonus) Step 4 - Look for signs in the portal that a failover occurred**   

You might be wondering, if there's a way to check if potentially a failover occurred. There is no clear "Failover occurred" message that exists today, however, checking the Resource Health can be a good indicator.  

In the Azure portal, navigate to your Azure SQL Database, and in the left-hand menu, under "Support + troubleshooting", select **Resource Health**. Some time after a failover (can be 5-15 minutes), you may see a health event similar to below. This can indicate several things, but one is that something has happened and Azure has decided to failover. 

![](./graphics/healthhistory.png)